In [1]:
from time import time, sleep
from IPython.display import Audio, display

def allDone():
  display(Audio(url='https://sound.peal.io/ps/audios/000/000/537/original/woo_vu_luvub_dub_dub.wav', autoplay=True))


In [2]:
spark.stop()
sc.stop()
conf = SparkConf().setAppName("jupyter: Analyze Twitter")
conf = conf.setMaster("spark://david-hdp:7077")
sc = SparkContext(conf=conf)
spark = SparkSession(sc)
spark

# Carga los tweets

In [5]:
df = spark.read.json('/user/david/flume/tweets/*')
df.count()

7.58


In [8]:
df.printSchema()

root
 |-- contributors: string (nullable = true)
 |-- coordinates: struct (nullable = true)
 |    |-- coordinates: array (nullable = true)
 |    |    |-- element: double (containsNull = true)
 |    |-- type: string (nullable = true)
 |-- created_at: string (nullable = true)
 |-- display_text_range: array (nullable = true)
 |    |-- element: long (containsNull = true)
 |-- entities: struct (nullable = true)
 |    |-- hashtags: array (nullable = true)
 |    |    |-- element: struct (containsNull = true)
 |    |    |    |-- indices: array (nullable = true)
 |    |    |    |    |-- element: long (containsNull = true)
 |    |    |    |-- text: string (nullable = true)
 |    |-- media: array (nullable = true)
 |    |    |-- element: struct (containsNull = true)
 |    |    |    |-- description: string (nullable = true)
 |    |    |    |-- display_url: string (nullable = true)
 |    |    |    |-- expanded_url: string (nullable = true)
 |    |    |    |-- id: long (nullable = true)
 |    |    |

In [9]:
df.registerTempTable("tweets")

# Se busca una matriz con el pais, el idiomar y los lenguajes mencionados

In [6]:
result = spark.sql("SELECT * \
                   FROM \
                           (SELECT \
                            place.country_code AS country, \
                            lang, \
                            IF(LOCATE(' javascript ', text)>0, 1, 0) AS javascript, \
                            IF(LOCATE(' c# ', text)>0, 1, 0) AS cSharp, \
                            IF(LOCATE(' java ', text)>0, 1, 0) AS java, \
                            IF(LOCATE(' android ', text)>0, 1, 0) AS android, \
                            IF(LOCATE(' python ', text)>0, 1, 0) AS python, \
                            IF(LOCATE(' c++ ', text)>0, 1, 0) AS cplus, \
                            IF(LOCATE(' php ', text)>0, 1, 0) AS php, \
                            IF(LOCATE(' ios ', text)>0, 1, 0) AS ios, \
                            IF(LOCATE(' c ', text)>0, 1, 0) AS c, \
                            IF(LOCATE(' html ', text)>0, 1, 0) AS html \
                        FROM tweets \
                        WHERE \
                            text NOT LIKE 'RT @%' \
                            AND place.country != 'None' \
                        ) AS filter \
                    WHERE \
                        javascript > 0 \
                        OR cSharp > 0 \
                        OR java > 0 \
                        OR android > 0 \
                        OR python > 0 \
                        OR cplus > 0 \
                        OR php > 0 \
                        OR c > 0 \
                        OR html > 0 ")

result.show()

618
+-------+----+----------+------+----+-------+------+-----+---+---+---+----+
|country|lang|javascript|cSharp|java|android|python|cplus|php|ios|  c|html|
+-------+----+----------+------+----+-------+------+-----+---+---+---+----+
|     US|  en|         0|     0|   0|      0|     0|    0|  0|  0|  1|   0|
|     FR|  fr|         0|     0|   0|      0|     0|    0|  0|  0|  1|   0|
|     FR|  fr|         0|     0|   0|      0|     0|    0|  0|  0|  1|   0|
|     BR|  pt|         0|     0|   0|      0|     0|    0|  0|  0|  1|   0|
|     FR|  fr|         0|     0|   0|      0|     0|    0|  0|  0|  1|   0|
|     BR|  pt|         0|     0|   0|      0|     0|    0|  0|  0|  1|   0|
|     BR|  pt|         0|     0|   0|      0|     0|    0|  0|  0|  1|   0|
|     FR|  fr|         0|     0|   0|      0|     0|    0|  0|  0|  1|   0|
|     BR|  pt|         0|     0|   0|      0|     0|    0|  0|  0|  1|   0|
|     FR|  fr|         0|     0|   0|      0|     0|    0|  0|  0|  1|   0|
|     FR

In [7]:
result = spark.sql("SELECT * \
                   FROM \
                           (SELECT \
                            lang, \
                            IF(LOCATE(' javascript ', text)>0, 1, 0) AS javascript, \
                            IF(LOCATE(' c# ', text)>0, 1, 0) AS cSharp, \
                            IF(LOCATE(' java ', text)>0, 1, 0) AS java, \
                            IF(LOCATE(' android ', text)>0, 1, 0) AS android, \
                            IF(LOCATE(' python ', text)>0, 1, 0) AS python, \
                            IF(LOCATE(' c++ ', text)>0, 1, 0) AS cplus, \
                            IF(LOCATE(' php ', text)>0, 1, 0) AS php, \
                            IF(LOCATE(' ios ', text)>0, 1, 0) AS ios, \
                            IF(LOCATE(' c ', text)>0, 1, 0) AS c, \
                            IF(LOCATE(' html ', text)>0, 1, 0) AS html \
                        FROM tweets \
                        WHERE \
                            text NOT LIKE 'RT @%' \
                        ) AS filter \
                    WHERE \
                        javascript > 0 \
                        OR cSharp > 0 \
                        OR java > 0 \
                        OR android > 0 \
                        OR python > 0 \
                        OR cplus > 0 \
                        OR php > 0 \
                        OR c > 0 \
                        OR html > 0 ")
result
print(result.count())
result.show()

9239
+----+----------+------+----+-------+------+-----+---+---+---+----+
|lang|javascript|cSharp|java|android|python|cplus|php|ios|  c|html|
+----+----------+------+----+-------+------+-----+---+---+---+----+
|  en|         0|     0|   0|      0|     0|    0|  0|  0|  1|   0|
|  en|         0|     0|   0|      0|     0|    0|  0|  0|  1|   0|
|  fr|         0|     0|   0|      0|     0|    0|  0|  0|  1|   0|
|  fr|         0|     0|   0|      0|     0|    0|  0|  0|  1|   0|
|  tl|         0|     0|   0|      0|     0|    0|  0|  0|  1|   0|
|  en|         0|     0|   0|      0|     0|    0|  0|  0|  1|   0|
|  fr|         0|     0|   0|      0|     0|    0|  0|  0|  1|   0|
|  pt|         0|     0|   0|      0|     0|    0|  0|  0|  1|   0|
|  fr|         0|     0|   0|      0|     0|    0|  0|  0|  1|   0|
|  tl|         0|     0|   0|      1|     0|    0|  0|  0|  0|   0|
|  fr|         0|     0|   0|      0|     0|    0|  0|  0|  1|   0|
|  en|         0|     0|   0|      1|     0

In [14]:
# export Results
#result.limit(1).coalesce(1).write.csv('carto.csv', header=True)

# Se agrupa la matriz por pais y se exporta

In [32]:
result.groupBy(result.country).sum()

DataFrame[country: string, sum(javascript): bigint, sum(cSharp): bigint, sum(java): bigint, sum(android): bigint, sum(python): bigint, sum(cplus): bigint, sum(php): bigint, sum(ios): bigint, sum(c): bigint, sum(html): bigint]

In [ ]:
   
    #Comrpueba si existe el fichero (0) o si no existe (1)
    check_geo="hdfs dfs -test -e twitter/single/geo.tweets.csv"
    #Crea un csv con todos los datos de la carpeta twitter/geo/*
    create_geo="hdfs dfs -getmerge twitter/geo/* geo.csv"
    #Añade cada linea del fichero geo.csv al fichero geo.tweets.csv
    merge_geo_local="cat geo.csv >> geo.tweets.csv"
    #Suma el fichero 'geo.csv' del local al 'twitter/single/geo.tweets.csv' del hdfs
    merge_geo_dfs="hdfs dfs -appendToFile geo.csv twitter/single/geo.tweets.csv"
    #Sube el fichero a hdfs
    upload_geo="hdfs dfs -put geo.tweets.csv twitter/single/"

    #Elimina todos los ficheros generados durante el proceso
    remove_resources_geo="hdfs dfs -rm -r twitter/geo/*"
    remove_geo_local="rm geo.csv"
    remove_geo_tweets="rm geo.tweets.csv"
    

        echo "Evaluando fichero geo"
        if [ '$exist_geo' == '1' ]; then
            echo "No existe fichero ../geo.tweets.csv"
            echo "Creando cabecera de csv"
            echo 'country, lang,javascript,cSharp,java,android,python,                          \
                cplus,php,ios,c,html' > geo.tweets.csv
            echo "Exportando datos del hdfs"
            eval $create_geo
            echo "Creando fichero local con cabecera"
            eval $merge_geo_local
            echo "Subiendo fichero ../geo.tweets.csv"
            eval $upload_geo
            echo "fichero ../geo.tweets.csv subido"
            echo "Eliminando ficheros geo auxiliares..."
            eval $remove_geo_local
            eval $remove_geo_tweets
            echo "Ficheros auxilaires geo, eliminados!"
        else
            echo "Existe el fichero ../geo.tweets.csv"
            echo "Exportando datos del hdfs"
            eval $create_geo
            echo "Uniendo fichero local con hdfs"
            eval $merge_geo_dfs
            echo "Eliminando fichero geo auxiliar..."
            eval $remove_geo_local
            echo "Fichero geo auxiliar, eliminado!"
        fi


    
    